In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
datasets = [train_data, test_data]
#drop columns with too many missing values and columns with no impact on the survival status
drop_columns = ["PassengerId", "Ticket", "Cabin"]
train_data.drop(drop_columns, axis = 1, inplace = True)


In [5]:
#check to see if there is any missing values in each data set
print("sum of missing values in train data is \n",train_data.isnull().sum() )
print("sum of missing values in test data is \n", test_data.isnull().sum())

sum of missing values in train data is 
 Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64
sum of missing values in test data is 
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [6]:
#replace missing values in 'Age', 'Embarked' and 'Fare'
for dataset in datasets:
    dataset.Age.fillna(dataset.Age.median(), inplace = True)
    dataset.Embarked.fillna(dataset.Embarked.mode()[0], inplace = True)
    dataset.Fare.fillna(dataset.Fare.median(),inplace = True)
    
#check to see if there is any missing values left in each data set
print("sum of missing values in train data is \n",train_data.isnull().sum())
print("sum of missing values in test data is \n", test_data.isnull().sum())



sum of missing values in train data is 
 Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
sum of missing values in test data is 
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


In [7]:

#feature engineering for train and test data
for dataset in datasets:
    #add a feature as 'Title' from each person in the shipinto the dataset  
    dataset["Title"] = dataset.Name.str.split(',',expand = True)[1].str.split('.',expand = True)[0]
    #add a feature as 'Family size' into the dataset
    dataset["Family_size"] = dataset["Parch"]+dataset["SibSp"]+1
    #add a feature as 'is along' = 0 if family size greater than 1, else equals to 1
    dataset["Is_along"] = 1
    dataset["Is_along"].loc[dataset["Family_size"]>1] = 0
    #add a bin for Fare
    dataset["Farebin"] = pd.qcut(dataset["Fare"],4)
    #add a bin for Age
    dataset["Agebin"] = pd.cut(dataset["Age"].astype(int), 5)
    
    


/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
#cleanup rare title names(with count value less than 10)
title_names = train_data["Title"].value_counts() < 10 #this will create a true false series with title name as index
train_data["Title"] = train_data["Title"].apply(lambda x : 'Misc' if (title_names).loc[x] == True else x)
print(train_data["Title"].value_counts())


 Mr        517
 Miss      182
 Mrs       125
 Master     40
Misc        27
Name: Title, dtype: int64


In [9]:
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_size,Is_along,Farebin,Agebin
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,Mr,2,0,"(-0.001, 7.91]","(16.0, 32.0]"
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,Mrs,2,0,"(31.0, 512.329]","(32.0, 48.0]"
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,Miss,1,1,"(7.91, 14.454]","(16.0, 32.0]"
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,Mrs,2,0,"(31.0, 512.329]","(32.0, 48.0]"
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,Mr,1,1,"(7.91, 14.454]","(32.0, 48.0]"


In [10]:
import sklearn
print(train_data.columns)
#encoding the categorical value
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
encoder = LabelEncoder()
for dataset in datasets:
    dataset["Embarked_encode"] = encoder.fit_transform(dataset["Embarked"])
    dataset["Sex_encode"]= encoder.fit_transform(dataset["Sex"]) 
    dataset["Title_encode"] = encoder.fit_transform(dataset["Title"])
    dataset["Agebin_encode"] = encoder.fit_transform(dataset["Agebin"])
    dataset["Farebin_encode"] = encoder.fit_transform(dataset["Farebin"])



Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'Title', 'Family_size', 'Is_along', 'Farebin', 'Agebin'],
      dtype='object')


In [11]:
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_size,Is_along,Farebin,Agebin,Embarked_encode,Sex_encode,Title_encode,Agebin_encode,Farebin_encode
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,Mr,2,0,"(-0.001, 7.91]","(16.0, 32.0]",2,1,2,1,0
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,Mrs,2,0,"(31.0, 512.329]","(32.0, 48.0]",0,0,3,2,3
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,Miss,1,1,"(7.91, 14.454]","(16.0, 32.0]",2,0,1,1,1
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,Mrs,2,0,"(31.0, 512.329]","(32.0, 48.0]",2,0,3,2,3
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,Mr,1,1,"(7.91, 14.454]","(32.0, 48.0]",2,1,2,2,1


In [12]:
train_data.head()
train_data.columns
test_data.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title', 'Family_size',
       'Is_along', 'Farebin', 'Agebin', 'Embarked_encode', 'Sex_encode',
       'Title_encode', 'Agebin_encode', 'Farebin_encode'],
      dtype='object')

In [13]:

feature_cols = ['Sex_encode', 'Embarked_encode',"Title_encode","Agebin_encode","Farebin_encode","Is_along", "Pclass"]
X_train = train_data[feature_cols]
y_train = train_data['Survived']
X_test = test_data[feature_cols]


MLA = [
    ensemble.RandomForestClassifier(),
    ensemble.GradientBoostingClassifier(),
    gaussian_process.GaussianProcessClassifier(),
    linear_model.LogisticRegressionCV(),
    tree.DecisionTreeClassifier(),
    XGBClassifier()
]

table_column = ['Model_name', 'Score']
table = pd.DataFrame(columns = table_column)
row_index = 0
for alg in MLA:
    table.loc[row_index, 'Model_name'] = alg.__class__.__name__
    fit_model = alg.fit(X_train, y_train)
    table.loc[row_index, 'Score'] = round(alg.score(X_train,y_train)*100,2)
    row_index +=1
table.sort_values(by = "Score", ascending = False, inplace = True)

table
    

,Model_name,Score
0,RandomForestClassifier,86.98
4,DecisionTreeClassifier,86.98
2,GaussianProcessClassifier,84.85
1,GradientBoostingClassifier,84.51
5,XGBClassifier,83.73
3,LogisticRegressionCV,79.57


In [14]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
print(acc_random_forest)
submission = pd.DataFrame({
        "PassengerId": test_data["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv("my_submission.csv",index=False)
#model = DecisionTreeClassifier()
#model.fit(X_train,y_train)
#predictions = model.predict(X_test)
#output = pd.DataFrame({'PassengerId':test_data.PassengerId,'Survived':predictions})
#output.to_csv('my_submission.csv',index=False)
#print("Your submission was successfully saved!")


86.98
